In [1]:
%matplotlib inline
import time
from notebook_init import *

out_root = Path('out/1dim')
makedirs(out_root, exist_ok=True)
torch.autograd.set_grad_enabled(True)
rand = lambda : np.random.randint(np.iinfo(np.int32).max)

In [70]:
use_w = True
dataset = 'ffhq'
inst = get_instrumented_model('StyleGAN2', dataset, 'style', device, inst=inst, use_w=use_w)
model = inst.model
model.truncation = 1.0

Loading ../models/checkpoints/stylegan2/stylegan2_ffhq_1024.pt


In [72]:
seeds_ffhq = [366745668]#, 327039870]#, 1302626592, 1235907362, 1150529896, 1881703227]
# seeds_car = [1839348078, 1150529896]#, 1881703227]
# seeds_cat = [889979887,263281582]
# seeds_ffhq = [1957070232, 946307783, 327039870]
# seeds_ffhq = [1302626592]#,327039870]
# seeds_ffhq = [rand() for _ in range(100)]
start_direction = 0
n_direction = 10
num_frames = 7
layer_mode = 'all'
perturb_intensity = 12

## Local Basis

In [82]:
save_img = True

for seed in seeds_ffhq:
    print(seed)
    rng = np.random.RandomState(seed)
    noise, z, z_local_basis, z_sv, noise_basis = get_random_local_basis(model, rng)
    lb_dir = z_local_basis.t().unsqueeze(1).detach().to(device) #reshape    
    layer_start, layer_end = LayerMode(layer_mode)    
    strips = []
    
    with torch.no_grad():
        for i in range(start_direction, start_direction + n_direction):
            batch_frames = create_strip_centered(inst, 'latent', 'style', [z], 0, lb_dir[i], 0, 1, 0, z, perturb_intensity, layer_start, layer_end, num_frames=num_frames, only_pos=0, only_neg=0)[0]
            strips.append(np.hstack(pad_frames(batch_frames)))
    grid = np.vstack(strips)
    
    if save_img:
        Image.fromarray(np.uint8(grid*255)).save(out_root / f'LocalBasis_{dataset}_{seed}_{start_direction}to{start_direction+n_direction-1}_ptb{perturb_intensity}.jpg')
    else:
        plt.figure(figsize=(20,40))
#         plt.figure(figsize=(10,20))
        plt.imshow(grid)
        plt.axis('off')
        plt.show()

366745668


# GANSpace - Only for FFHQ

In [74]:
gs_dir = np.load('./global_directions/ganspace_directions_ffhq_stylegan2.npy')#Note! Only ffhq is provided.
gs_dir = torch.from_numpy(gs_dir).to(device)

In [79]:
save_img = True

for seed in seeds_ffhq:
    rng = np.random.RandomState(seed)
    noise = torch.from_numpy(
            rng.standard_normal(512 * 1)
            .reshape(1, 512)).float().to(model.device) #[N, 512]
    if model.w_primary:
        z = model.model.style(noise)
    layer_start, layer_end = LayerMode(layer_mode)
    strips = []
    
    with torch.no_grad():
        for i in range(start_direction, start_direction + n_direction):
            batch_frames = create_strip_centered(inst, 'latent', 'style', [z], 0, gs_dir[i], 0, 1, 0, z, perturb_intensity, layer_start, layer_end, num_frames=num_frames, only_pos=0, only_neg=0)[0]
            strips.append(np.hstack(pad_frames(batch_frames)))
    grid = np.vstack(strips)
    
    if save_img:
        Image.fromarray(np.uint8(grid*255)).save(out_root / f'ganspace_{seed}_{start_direction}to{start_direction+n_direction-1}_ptb{perturb_intensity}.jpg')
    else:
        plt.figure(figsize=(20,40))
        plt.imshow(grid)
        plt.axis('off')
        plt.show()

# SeFa - Only for FFHQ

In [76]:
sf_dir = np.load('./global_directions/sefa_directions_ffhq.npy') #Note! Only ffhq is provided.
sf_dir = torch.from_numpy(sf_dir).to(device)

In [81]:
save_img = True

for seed in seeds_ffhq:
    rng = np.random.RandomState(seed)
    noise = torch.from_numpy(
            rng.standard_normal(512 * 1)
            .reshape(1, 512)).float().to(model.device) #[N, 512]
    if model.w_primary:
        z = model.model.style(noise)
    layer_start, layer_end = LayerMode(layer_mode)
    strips = []
    
    with torch.no_grad():
        for i in range(start_direction, start_direction + n_direction):
            batch_frames = create_strip_centered(inst, 'latent', 'style', [z], 0, sf_dir[i], 0, 1, 0, z, perturb_intensity, layer_start, layer_end, num_frames=num_frames, only_pos=0, only_neg=0)[0]
            strips.append(np.hstack(pad_frames(batch_frames)))
    grid = np.vstack(strips)
    
    if save_img:
        Image.fromarray(np.uint8(grid*255)).save(out_root / f'sefa_{seed}_{start_direction}to{start_direction+n_direction-1}_ptb{perturb_intensity}.jpg')
    else:
        plt.figure(figsize=(20,40))
        plt.imshow(grid)
        plt.axis('off')
        plt.show()